In [1]:
from requests import get
from bs4 import BeautifulSoup as bs

In [2]:
url = 'https://www.imdb.com/search/title/?release_date=2019&sort=num_votes,desc&page=1'
response = get(url)

In [3]:
html_soup =  bs(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [4]:
movie_arr = html_soup.find_all('div', class_ = 'lister-item mode-advanced')
print(type(movie_arr))
len(movie_arr)

<class 'bs4.element.ResultSet'>


50

total 50 div classes

In [5]:
names = []
years = []
imdb_ratings= []
metascores = []
votes =[]
genres = []

for item in movie_arr:
    if item.find('div', class_ = 'ratings-metascore') is not None:
# Name of movie
        name = item.h3.a.text
        names.append(name)
# Year of the movie
        yr = item.h3.find('span', class_ = 'lister-item-year').text
        years.append(yr)
# IMDB rating
        imdb = float(item.strong.text)
        imdb_ratings.append(imdb)
# Metascore
        ms = item.find('span',  class_ = 'metascore').text
        metascores.append(int(ms))
# Number of votes
        vote = item.find('span', attrs = {'name' : 'nv'})['data-value']
        votes.append(int(vote))
# Genre of the movie
        genre = item.find('span', class_ = 'genre').text
        genres.append(genre)

Saving all in a data frame

In [6]:
import pandas as pd
imdb_ = pd.DataFrame({'movie' : names,
                      'year' : years,
                      'rating' : imdb_ratings,
                      'metascore' : metascores,
                      'votes' : votes,
                      'genres' : genres
                     })
imdb_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movie      40 non-null     object 
 1   year       40 non-null     object 
 2   rating     40 non-null     float64
 3   metascore  40 non-null     int64  
 4   votes      40 non-null     int64  
 5   genres     40 non-null     object 
dtypes: float64(1), int64(2), object(3)
memory usage: 2.0+ KB


In [7]:
imdb_.head()

,movie,year,rating,metascore,votes,genres
0,Joker,(2019),8.5,59,856761,"\nCrime, Drama, Thriller"
1,Avengers: Endgame,(2019),8.4,78,755906,"\nAction, Adventure, Drama"
2,Once Upon a Time... in Hollywood,(2019),7.6,83,507965,"\nComedy, Drama"
3,Gisaengchung,(2019),8.6,96,472208,"\nComedy, Drama, Thriller"
4,Captain Marvel,(2019),6.9,64,422980,"\nAction, Adventure, Sci-Fi"


In [8]:
imdb_['year'].unique()

array(['(2019)', '(II) (2019)', '(I) (2019)'], dtype=object)

In [9]:
imdb_.loc[:,'year'] = imdb_['year'].str[-5:-1].astype(int)

In [10]:
imdb_['genres'] = imdb_['genres'].replace('\n',' ', regex=True)

In [11]:
imdb_.head()

,movie,year,rating,metascore,votes,genres
0,Joker,2019,8.5,59,856761,"Crime, Drama, Thriller"
1,Avengers: Endgame,2019,8.4,78,755906,"Action, Adventure, Drama"
2,Once Upon a Time... in Hollywood,2019,7.6,83,507965,"Comedy, Drama"
3,Gisaengchung,2019,8.6,96,472208,"Comedy, Drama, Thriller"
4,Captain Marvel,2019,6.9,64,422980,"Action, Adventure, Sci-Fi"


In [12]:
imdb_.to_csv('imdb_movie_ratings.csv')